In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../../Elenco SME_2025-2-ICMC.csv', delimiter=';')

In [4]:
df = df.dropna(subset=["Disciplina (código)"])

In [5]:
df = df.rename(columns={"Disciplina (código)": "codigo", 
                        "Disciplina (nome completo)": "nome", 
                        "Curso(s)": "curso",
                        "Horário 1": "horario1", "Horário 2": "horario2", "Ementa": "ementa", "Turma": "turma"})

In [6]:
df = df.astype({'turma': 'int32'})

In [7]:
df

,codigo,nome,curso,turma,horario1,horario2,nivel,ementa
0,SME0142,Algebra Linear e Aplicações,"BCC, BCDados",1,Terça - 10:10 / 11:50,Quinta - 10:10 / 11:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
1,SME0142,Algebra Linear e Aplicações,BCC,2,Terça - 08:10 / 09:50,Quinta - 08:10 / 09:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
2,SME0123,Estatística,BCC,1,Terça - 14:20 / 16:00,Sexta - 08:10 / 09:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
3,SME0123,Estatística,BCC,2,Terça - 16:20 / 18:00,Sexta - 10:10 / 11:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
4,SME0110,Programação Matemática,BCC,1,Segunda - 08:10 / 09:50,Quarta - 10:10 / 11:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
...,...,...,...,...,...,...,...,...
58,SME5962,Otimização Inteira,Otimização,1,a definir,NaN,p,https://icmc.usp.br/pos-graduacao/disciplinas?...
59,SMA5934,Tópicos em Teoria Algébrica de Invariantes e B...,Matemática,1,a definir,NaN,p,https://icmc.usp.br/pos-graduacao/disciplinas?...
60,SMA5802,Equações Diferenciais Ordinárias,Matemática,1,a definir,NaN,p,https://icmc.usp.br/pos-graduacao/disciplinas?...
61,SME5873,Ferramentas de Programação Paralela para Soluç...,MFC,1,a definir,NaN,p,https://icmc.usp.br/pos-graduacao/disciplinas?...


In [8]:
json_data = df.to_dict(orient='records')

In [9]:
for item in json_data:
    keys_to_remove = [k for k, v in item.items() if pd.isna(v) and k == "horario2"]
    for k in keys_to_remove:
        del item[k]

In [10]:
len(json_data)

63

In [11]:
dias_map = {
    "Segunda": "Seg.",
    "Terça": "Ter.",
    "Quarta": "Qua.",
    "Quinta": "Qui.",
    "Sexta": "Sex.",
    "Sábado": "Sáb.",
    "Domingo": "Dom."
}

In [12]:
def processar_horario(horario):
    if pd.isna(horario):
        return None
    try:
        # Exemplo: "Terça - 14:20 / 16:00"
        dia_e_horas = horario.split('-')
        dia_extenso = dia_e_horas[0].strip()
        dia = dias_map.get(dia_extenso, dia_extenso)  # Se não tiver no mapa, mantém original
        horas = dia_e_horas[1].strip().split('/')
        inicio = horas[0].strip()
        fim = horas[1].strip()
        return {"dia": dia, "inicio": inicio, "fim": fim}
    except Exception as e:
        print(f"Erro ao processar: {horario} -> {e}")
        return None

In [13]:
for item in json_data:
    horarios = []
    for key in ['horario1', 'horario2']:
        if key in item:
            h = processar_horario(item[key])
            if h:
                horarios.append(h)
            del item[key]  # Remove os campos antigos

    # if horarios:
    item['horarios'] = horarios
    item['id'] = f'{item['codigo'].rstrip().lstrip()},{item['turma']}'


Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: 
Sex. 10:10/11:50 -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a def

In [14]:
len(json_data)

63

In [15]:
from datetime import datetime

In [16]:
def hora_para_minutos(hora_str):
    """Converte 'HH:MM' para minutos desde 00:00"""
    h, m = map(int, hora_str.split(":"))
    return h * 60 + m

In [17]:
def horarios_conflitam(h1, h2):
    """Verifica se dois horários conflitam no mesmo dia"""
    if h1['dia'] != h2['dia']:
        return False

    inicio1 = hora_para_minutos(h1['inicio'])
    fim1 = hora_para_minutos(h1['fim'])
    inicio2 = hora_para_minutos(h2['inicio'])
    fim2 = hora_para_minutos(h2['fim'])

    return max(inicio1, inicio2) < min(fim1, fim2)

In [18]:
# 16:00 -> 18:00
a = {"inicio": "16:00", "fim": "18:00", "dia": "Seg."}
# 14:00 -> 16:00
b = {"inicio": "14:00", "fim": "16:01", "dia": "Seg."}

# 16:00 -> 18:00
a = {"inicio": "16:00", "fim": "18:00", "dia": "Seg."}
# 14:00 -> 16:00
b = {"inicio": "18:00", "fim": "19:01", "dia": "Seg."}

horarios_conflitam(a, b)

False

In [19]:
# Inicializa os sets de conflitos
for turma in json_data:
    turma['conflitos'] = set()

# Compara todas as turmas em pares (sem repetições)
for i in range(len(json_data)):
    turma_i = json_data[i]
    for j in range(i + 1, len(json_data)):
        turma_j = json_data[j]

        # Verifica se algum horário da turma_i conflita com algum da turma_j
        for h1 in turma_i.get('horarios', []):
            for h2 in turma_j.get('horarios', []):
                if horarios_conflitam(h1, h2):
                    turma_i['conflitos'].add(turma_j['id'])
                    turma_j['conflitos'].add(turma_i['id'])
                    break  # Um conflito já basta

# Converte os sets para listas (JSON serializável)
for turma in json_data:
    turma['conflitos'] = list(turma['conflitos'])

In [20]:
def marcar_turmas_noturnas(json_data):
    for turma in json_data:
        turma['noturna'] = False  # valor padrão
        for horario in turma.get('horarios', []):
            try:
                hora, minuto = map(int, horario['inicio'].split(':'))
                if hora >= 18:
                    turma['noturna'] = True
                    break  # Já é noturna, não precisa verificar os outros horários
            except Exception as e:
                print(f"Erro ao processar horário {horario['inicio']} da turma {turma.get('id')}: {e}")


In [21]:
marcar_turmas_noturnas(json_data)

In [22]:
json_data.sort(key=lambda x: x['codigo'])


In [23]:
import json

In [24]:
with open('turmas.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [25]:
json_data

[{'codigo': 'PRG-0027',
  'nome': 'Uma jornada lúdica pela topologia geométrica\t',
  'curso': 'Optativa Livre para toda USP',
  'turma': 1,
  'nivel': 'p',
  'ementa': 'https://uspdigital.usp.br/jupiterweb/obterDisciplina?sgldis=PRG-0027',
  'horarios': [{'dia': 'Ter.', 'inicio': '16:20', 'fim': '18:00'}],
  'id': 'PRG-0027,1',
  'conflitos': ['SME0220,1',
   'SME0123,2',
   'SME0332,2',
   'SME0285,1',
   'SME0320,2'],
  'noturna': False},
 {'codigo': 'SMA5802',
  'nome': 'Equações Diferenciais Ordinárias',
  'curso': 'Matemática',
  'turma': 1,
  'nivel': 'p',
  'ementa': 'https://icmc.usp.br/pos-graduacao/disciplinas?programa=55134&disciplina=SMA5802',
  'horarios': [],
  'id': 'SMA5802,1',
  'conflitos': [],
  'noturna': False},
 {'codigo': 'SMA5934',
  'nome': 'Tópicos em Teoria Algébrica de Invariantes e Bifurcações',
  'curso': 'Matemática',
  'turma': 1,
  'nivel': 'p',
  'ementa': 'https://icmc.usp.br/pos-graduacao/disciplinas?programa=55134&disciplina=SMA5934',
  'horarios':